# Imports and setting up viz

NB : conda env1 on PC, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *
from tools_hf import *
from tools_native import *
from tools_LIAISE import *

# sys.path.append('../../python_tools/aborella/PLOTS/')
# from datasets import *
sys.path.append('../../python_tools/aborella/UTIL/')
import xr_utils

In [ ]:
psy.rcParams['auto_show'] = True
mpl.rcParams['figure.figsize'] = [10., 8.]

# Load model files and edit datasets

LAM output analysis.

Simu irr et simu no-irr.

In [ ]:
# nat_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/hourly_native'
nat_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/halfhour_native'

## LMDZ

In [ ]:
# def add_relative_humidity(ds):
#     if not all(var in ds for var in ['temp', 'ovap']):
#         raise ValueError("Dataset must contain 'temp' (temperature) and 'ovap' (specific humidity).")
#     if 'presnivs' not in ds.coords:
#         raise ValueError("Dataset must have 'presnivs' as a coordinate.")

#     # Constants
#     Rd = 287.05  # Gas constant for dry air (J/(kg·K))
#     Rv = 461.5   # Gas constant for water vapor (J/(kg·K))
#     eps = Rd / Rv  # Ratio of gas constants

#     # Extract variables
#     temp = ds['temp']  # Temperature in °C, shape (time, presnivs, cell)
#     ovap = ds['ovap']  # Specific humidity in kg/kg, shape (time, presnivs, cell)
#     presnivs = ds.coords['presnivs']  # Pressure levels in hPa, shape (presnivs,)

#     # Broadcast pressure levels to match the shape of temp and ovap
#     presnivs_broadcasted = xr.broadcast(temp, presnivs)[1]

#     # Compute saturation vapor pressure (hPa) and actual vapor pressure (hPa)
#     e_s = 6.112 * np.exp((17.67 * temp) / (temp + 243.5))
#     e = (ovap * presnivs_broadcasted) / (eps + (1 - eps) * ovap)

#     # Compute relative humidity (%)
#     ds['rh'] = (e / e_s) * 100
#     ds['rh'].attrs['units'] = '%'
#     ds['rh'].attrs['description'] = 'Relative Humidity based on temp, ovap, and presnivs'

#     return ds

def add_relative_humidity(ds):
    if not all(var in ds for var in ['temp', 'ovap']):
        raise ValueError("Dataset must contain 'temp' (temperature) and 'ovap' (specific humidity).")
    if 'presnivs' not in ds.coords:
        raise ValueError("Dataset must have 'presnivs' as a coordinate.")

    # Constants
    Rd = 287.05
    Rv = 461.5
    eps = Rd / Rv

    # Compute relative humidity directly
    ds['pressure'] = ds['presnivs'].broadcast_like(ds['temp'])
    ds['rh'] = (
        (ds['ovap'] * ds['pressure']) / (eps + (1 - eps) * ds['ovap']) / 
        (6.112 * np.exp((17.67 * ds['temp']) / (ds['temp'] + 243.5))) * 100
    )

    ds['rh'].attrs['units'] = '%'
    ds['rh'].attrs['description'] = 'Relative Humidity based on temp, ovap, and presnivs'

    return ds

In [ ]:
def format_lmdz_HF(filename, color, name):
    ds = xr.open_mfdataset(filename)
    ds = ds.rename({'time_counter':'time'})
    ds = ds.assign_coords(time_decimal=ds.time.dt.hour + ds.time.dt.minute / 60)
    ds.attrs['name'] = name
    ds.attrs['plot_color']=color

    ds['sens']=-ds['sens']
    ds['flat']=-ds['flat']

    ds['ground_level'] = ds['phis'] / 9.81
    ds['ground_level'].attrs['units'] = 'm'

    ds=add_wind_speed(ds)
    ds=add_wind_direction(ds)
    ds=add_wind_10m(ds)
    ds=add_relative_humidity(ds)

    #make ovap unit g/kg
    ds['ovap'] = ds['ovap']*1000
    ds['ovap'].attrs['units'] = 'g/kg'
    ds['ovap'].attrs['long_name'] = 'Specific humidity'
    #same for q2m
    ds['q2m'] = ds['q2m']*1000
    ds['q2m'].attrs['units'] = 'g/kg'
    #turn psol to hPa
    ds['psol'] = ds['psol']/100
    ds['psol'].attrs['units'] = 'hPa'
    #make precip unit mm/30mn (like obs)
    ds['precip']=ds['precip'] * 60 * 30
    ds['precip'].attrs['long_name'] = 'Total precipitation'
    ds['precip'].attrs['units']='mm (over 30mn)'

    
    return ds

In [ ]:
lmdz_noirr_file = '{}/sim267_20210701_20210731_HF_histhf.nc'.format(nat_dir)
lmdz_noirr = format_lmdz_HF(lmdz_noirr_file, 'red', 'noirr')
lmdz_noirr

In [ ]:
lmdz_irr_file = '{}/sim268_20210701_20210731_HF_histhf.nc'.format(nat_dir)
lmdz_irr = format_lmdz_HF(lmdz_irr_file, 'blue', 'irr')
lmdz_irr

## ORCHIDEE

In [ ]:
def format_ORC_HF(filename, color, name):
    ds = xr.open_mfdataset(filename)
    ds = ds.rename({'time_counter':'time'})
    ds = ds.assign_coords(time_decimal=ds.time.dt.hour + ds.time.dt.minute / 60)
    ds.attrs['name'] = name
    ds.attrs['plot_color']= color
    # ds = ds.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

    ds['evapot']=ds['evapot'] * 3600 * 24
    ds['evapot'].attrs['units'] = 'mm/d'

    ds['soil_temp_01'] = ds['ptn'][:,3,:] - 273.15
    ds['soil_temp_01'].attrs['units'] = '°C'
    ds['soil_temp_01'].attrs['long_name'] = 'Soil temperature at 1.3cm'
    ds['soil_temp_03'] = ds['ptn'][:,4,:] - 273.15
    ds['soil_temp_03'].attrs['units'] = '°C'
    ds['soil_temp_03'].attrs['long_name'] = 'Soil temperature at 2.9cm'
    ds['soil_temp_06'] = ds['ptn'][:,5,:] - 273.15
    ds['soil_temp_06'].attrs['units'] = '°C'
    ds['soil_temp_06'].attrs['long_name'] = 'Soil temperature at 6.06cm'
    ds['soil_temp_12'] = ds['ptn'][:,6,:] - 273.15
    ds['soil_temp_12'].attrs['units'] = '°C'
    ds['soil_temp_12'].attrs['long_name'] = 'Soil temperature at 12.3cm'

    return ds

In [ ]:
orc_noirr_file = '{}/sim267_20210701_20210731_HF_sechiba_out_2.nc'.format(nat_dir)
orc_noirr = format_ORC_HF(orc_noirr_file, 'red', 'noirr')
orc_noirr

In [ ]:
orc_irr_file =  '{}/sim268_20210701_20210731_HF_sechiba_out_2.nc'.format(nat_dir)
orc_irr = format_ORC_HF(orc_irr_file, 'blue', 'irr')
orc_irr

## Selecting subdatasets

### LIAISE sites

In [ ]:
#on grid cell
#highirr
Cendrosa_lon=0.755
Cendrosa_lat=41.63
#lowirr
ElsPlans_lon=0.996
ElsPlans_lat=41.51

# alt for these grid cells : (307.6005859375, 456.8945007324219)

In [ ]:
#actual data from LIAISE db
Cendrosa_Latitude=41.69336
Cendrosa_Longitude=0.928538
Cenrosa_altitude= 240

ElsPlans_Latitude = 41.590111
ElsPlans_Longitude = 1.029363
ElsPlans_altitude = 334

In [ ]:
cen_lmdz_noirr = sel_closest(lmdz_noirr, Cendrosa_lon, Cendrosa_lat)
cen_lmdz_irr = sel_closest(lmdz_irr, lon=Cendrosa_lon,lat=Cendrosa_lat)
cen_orc_noirr = sel_closest(orc_noirr, lon=Cendrosa_lon,lat=Cendrosa_lat)
cen_orc_irr = sel_closest(orc_irr, lon=Cendrosa_lon,lat=Cendrosa_lat)
cen_lmdz_noirr.attrs['name']='La Cendrosa no_irr'
cen_lmdz_irr.attrs['name']='La Cendrosa irr'
cen_orc_noirr.attrs['name']='La Cendrosa no_irr'
cen_orc_irr.attrs['name']='La Cendrosa irr'

In [ ]:
els_lmdz_noirr = sel_closest(lmdz_noirr, lon=ElsPlans_lon,lat=ElsPlans_lat)
els_lmdz_irr   = sel_closest(lmdz_irr, lon=ElsPlans_lon,lat=ElsPlans_lat)
els_orc_noirr  = sel_closest(orc_noirr, lon=ElsPlans_lon,lat=ElsPlans_lat)
els_orc_irr    = sel_closest(orc_irr, lon=ElsPlans_lon,lat=ElsPlans_lat)
els_lmdz_irr.attrs['name']='Els Plans irr'
els_orc_noirr.attrs['name']='Els Plans no_irr'
els_lmdz_noirr.attrs['name']='Els Plans no_irr'
els_orc_irr.attrs['name']='Els Plans irr'

In [ ]:
#add attr linestyle -- to els_ datasets
els_lmdz_noirr.attrs['linestyle']='--'
els_lmdz_irr.attrs['linestyle']='--'
els_orc_noirr.attrs['linestyle']='--'
els_orc_irr.attrs['linestyle']='--'
#add attr linestyle - to cen_ datasets
cen_lmdz_noirr.attrs['linestyle']='-'
cen_lmdz_irr.attrs['linestyle']='-'
cen_orc_noirr.attrs['linestyle']='-'
cen_orc_irr.attrs['linestyle']='-'

In [ ]:
cendrosa_gl= cen_lmdz_noirr['ground_level'].mean(dim='time').values.item()
elsplans_gl= els_lmdz_noirr['ground_level'].mean(dim='time').values.item()
cendrosa_gl,  elsplans_gl

In [ ]:
def add_var_on_mast_level(ds: xr.Dataset, var: str, height:int, newvar_name: str, unit: str, longname: str) -> xr.Dataset:
    # Compute closest index to height
    closest_idx = np.abs(ds["geoph"] - ds['ground_level'] - height).argmin(dim="presnivs").compute()

    # Assign new variable with proper metadata
    ds[newvar_name] = ds[var].isel(presnivs=closest_idx)
    ds[newvar_name].attrs.update({"long_name" : longname, "units": unit})
    
    return ds

In [ ]:
mast_temps=True
mast_temps=False

mast_q=True
mast_q=False

mast_rh=True
mast_rh=False

mast_fluxes=True
mast_fluxes=False

In [ ]:
if mast_temps:
    cen_lmdz_noirr = add_var_on_mast_level(cen_lmdz_noirr, 'temp', 10, 't10m', 'K', 'Temperature at 10m')
    cen_lmdz_irr   = add_var_on_mast_level(cen_lmdz_irr,   'temp', 10, 't10m', 'K', 'Temperature at 10m')
    els_lmdz_noirr = add_var_on_mast_level(els_lmdz_noirr, 'temp', 10, 't10m', 'K', 'Temperature at 10m')
    els_lmdz_irr   = add_var_on_mast_level(els_lmdz_irr,   'temp', 10, 't10m', 'K', 'Temperature at 10m')

    cen_lmdz_noirr = add_var_on_mast_level(cen_lmdz_noirr, 'temp', 25, 't25m', 'K', 'Temperature at 25m')
    cen_lmdz_irr   = add_var_on_mast_level(cen_lmdz_irr,   'temp', 25, 't25m', 'K', 'Temperature at 25m')
    els_lmdz_noirr = add_var_on_mast_level(els_lmdz_noirr, 'temp', 25, 't25m', 'K', 'Temperature at 25m')
    els_lmdz_irr   = add_var_on_mast_level(els_lmdz_irr,   'temp', 25, 't25m', 'K', 'Temperature at 25m')

    cen_lmdz_noirr = add_var_on_mast_level(cen_lmdz_noirr, 'temp', 50, 't50m', 'K', 'Temperature at 50m')
    cen_lmdz_irr   = add_var_on_mast_level(cen_lmdz_irr,   'temp', 50, 't50m', 'K', 'Temperature at 50m')
    els_lmdz_noirr = add_var_on_mast_level(els_lmdz_noirr, 'temp', 50, 't50m', 'K', 'Temperature at 50m')
    els_lmdz_irr   = add_var_on_mast_level(els_lmdz_irr,   'temp', 50, 't50m', 'K', 'Temperature at 50m')

In [ ]:
if mast_q:
    cen_lmdz_noirr = add_var_on_mast_level(cen_lmdz_noirr, 'ovap', 10, 'q10m', 'K', 'q at 10m')
    cen_lmdz_irr   = add_var_on_mast_level(cen_lmdz_irr,   'ovap', 10, 'q10m', 'K', 'q at 10m')
    els_lmdz_noirr = add_var_on_mast_level(els_lmdz_noirr, 'ovap', 10, 'q10m', 'K', 'q at 10m')
    els_lmdz_irr   = add_var_on_mast_level(els_lmdz_irr,   'ovap', 10, 'q10m', 'K', 'q at 10m')
    cen_lmdz_noirr = add_var_on_mast_level(cen_lmdz_noirr, 'ovap', 25, 'q25m', 'K', 'q at 25m')
    cen_lmdz_irr   = add_var_on_mast_level(cen_lmdz_irr,   'ovap', 25, 'q25m', 'K', 'q at 25m')
    els_lmdz_noirr = add_var_on_mast_level(els_lmdz_noirr, 'ovap', 25, 'q25m', 'K', 'q at 25m')
    els_lmdz_irr   = add_var_on_mast_level(els_lmdz_irr,   'ovap', 25, 'q25m', 'K', 'q at 25m')
    cen_lmdz_noirr = add_var_on_mast_level(cen_lmdz_noirr, 'ovap', 50, 'q50m', 'K', 'q at 50m')
    cen_lmdz_irr   = add_var_on_mast_level(cen_lmdz_irr,   'ovap', 50, 'q50m', 'K', 'q at 50m')
    els_lmdz_noirr = add_var_on_mast_level(els_lmdz_noirr, 'ovap', 50, 'q50m', 'K', 'q at 50m')
    els_lmdz_irr   = add_var_on_mast_level(els_lmdz_irr,   'ovap', 50, 'q50m', 'K', 'q at 50m')

In [ ]:
if mast_rh:
    cen_lmdz_noirr = add_var_on_mast_level(cen_lmdz_noirr, 'rh', 10, 'rh10m', 'K', 'rh at 10m')
    cen_lmdz_irr   = add_var_on_mast_level(cen_lmdz_irr,   'rh', 10, 'rh10m', 'K', 'rh at 10m')
    els_lmdz_noirr = add_var_on_mast_level(els_lmdz_noirr, 'rh', 10, 'rh10m', 'K', 'rh at 10m')
    els_lmdz_irr   = add_var_on_mast_level(els_lmdz_irr,   'rh', 10, 'rh10m', 'K', 'rh at 10m')
    cen_lmdz_noirr = add_var_on_mast_level(cen_lmdz_noirr, 'rh', 25, 'rh25m', 'K', 'rh at 25m')
    cen_lmdz_irr   = add_var_on_mast_level(cen_lmdz_irr,   'rh', 25, 'rh25m', 'K', 'rh at 25m')
    els_lmdz_noirr = add_var_on_mast_level(els_lmdz_noirr, 'rh', 25, 'rh25m', 'K', 'rh at 25m')
    els_lmdz_irr   = add_var_on_mast_level(els_lmdz_irr,   'rh', 25, 'rh25m', 'K', 'rh at 25m')
    cen_lmdz_noirr = add_var_on_mast_level(cen_lmdz_noirr, 'rh', 50, 'rh50m', 'K', 'rh at 50m')
    cen_lmdz_irr   = add_var_on_mast_level(cen_lmdz_irr,   'rh', 50, 'rh50m', 'K', 'rh at 50m')
    els_lmdz_noirr = add_var_on_mast_level(els_lmdz_noirr, 'rh', 50, 'rh50m', 'K', 'rh at 50m')
    els_lmdz_irr   = add_var_on_mast_level(els_lmdz_irr,   'rh', 50, 'rh50m', 'K', 'rh at 50m')

### Golden days

In [ ]:
start_time = '2021-07-21T00:30:00.000000000'
day1_end_time = '2021-07-21T23:30:00.000000000'
day2_start_time = '2021-07-22T00:30:00.000000000'
end_time='2021-07-22T23:30:00.000000000'

In [ ]:
gold_cen_lmdz_noirr = filter_xarray_by_timestamps(cen_lmdz_noirr, start_time, end_time)
gold_cen_lmdz_irr = filter_xarray_by_timestamps(cen_lmdz_irr, start_time, end_time)
gold_cen_orc_noirr = filter_xarray_by_timestamps(cen_orc_noirr, start_time, end_time)
gold_cen_orc_irr = filter_xarray_by_timestamps(cen_orc_irr, start_time, end_time)

gold_els_lmdz_noirr = filter_xarray_by_timestamps(els_lmdz_noirr, start_time, end_time)
gold_els_lmdz_irr = filter_xarray_by_timestamps(els_lmdz_irr, start_time, end_time)
gold_els_orc_noirr = filter_xarray_by_timestamps(els_orc_noirr, start_time, end_time)
gold_els_orc_irr = filter_xarray_by_timestamps(els_orc_irr, start_time, end_time)

In [ ]:
#day1
day1_cen_lmdz_noirr = filter_xarray_by_timestamps(cen_lmdz_noirr, start_time, day1_end_time)
day1_cen_lmdz_irr = filter_xarray_by_timestamps(cen_lmdz_irr, start_time, day1_end_time)
day1_cen_orc_noirr = filter_xarray_by_timestamps(cen_orc_noirr, start_time, day1_end_time)
day1_cen_orc_irr = filter_xarray_by_timestamps(cen_orc_irr, start_time, day1_end_time)

day1_els_lmdz_noirr = filter_xarray_by_timestamps(els_lmdz_noirr, start_time, day1_end_time)
day1_els_lmdz_irr = filter_xarray_by_timestamps(els_lmdz_irr, start_time, day1_end_time)
day1_els_orc_noirr = filter_xarray_by_timestamps(els_orc_noirr, start_time, day1_end_time)
day1_els_orc_irr = filter_xarray_by_timestamps(els_orc_irr, start_time, day1_end_time)

#day2
day2_cen_lmdz_noirr = filter_xarray_by_timestamps(cen_lmdz_noirr, day2_start_time, end_time)
day2_cen_lmdz_irr = filter_xarray_by_timestamps(cen_lmdz_irr, day2_start_time, end_time)
day2_cen_orc_noirr = filter_xarray_by_timestamps(cen_orc_noirr, day2_start_time, end_time)
day2_cen_orc_irr = filter_xarray_by_timestamps(cen_orc_irr, day2_start_time, end_time)

day2_els_lmdz_noirr = filter_xarray_by_timestamps(els_lmdz_noirr, day2_start_time, end_time)
day2_els_lmdz_irr = filter_xarray_by_timestamps(els_lmdz_irr, day2_start_time, end_time)
day2_els_orc_noirr = filter_xarray_by_timestamps(els_orc_noirr, day2_start_time, end_time)
day2_els_orc_irr = filter_xarray_by_timestamps(els_orc_irr, day2_start_time, end_time)

## Monthly files for July 2021

In [ ]:
lmdz_noirr_1M_file = '{}/sim267_20210701_20210731_1M_histmth.nc'.format(nat_dir)
lmdz_noirr_1M = format_lmdz_HF(lmdz_noirr_1M_file, 'red', 'noirr')

lmdz_irr_1M_file = '{}/sim268_20210701_20210731_1M_histmth.nc'.format(nat_dir)
lmdz_irr_1M = format_lmdz_HF(lmdz_irr_1M_file, 'blue', 'irr')

lmdz_irr_1M

In [ ]:
orc_noirr_1M_file =  '{}/sim267_20210701_20210731_1M_sechiba_history.nc'.format(nat_dir)
orc_noirr_1M = format_ORC_HF(orc_noirr_1M_file, 'red', 'noirr')

orc_irr_1M_file =  '{}/sim268_20210701_20210731_1M_sechiba_history.nc'.format(nat_dir)
orc_irr_1M = format_ORC_HF(orc_irr_1M_file, 'blue', 'irr')
orc_irr_1M['irrig_frac'] = orc_irr_1M['irrigmap_dyn']/orc_irr_1M['Areas'] * 100

orc_noirr_1M

## Roughdyn=N

In [ ]:
lmdz_noroughdyn_file = '{}/sim269_20210701_20210731_HF_histhf.nc'.format(nat_dir)
lmdz_noroughdyn = format_lmdz_HF(lmdz_noroughdyn_file, 'green', 'noirr_noroughdyn')

orc_noroughdyn_file =  '{}/sim269_20210701_20210731_HF_sechiba_out_2.nc'.format(nat_dir)
orc_noroughdyn = format_ORC_HF(orc_noroughdyn_file, 'green', 'noirr_noroughdyn')

In [ ]:
lmdz_noroughdyn_1M_file = '{}/sim269_20210701_20210731_1M_histmth.nc'.format(nat_dir)
lmdz_noroughdyn_1M = format_lmdz_HF(lmdz_noroughdyn_1M_file, 'green', 'noirr_noroughdyn')

orc_noroughdyn_1M_file =  '{}/sim269_20210701_20210731_1M_sechiba_history.nc'.format(nat_dir)
orc_noroughdyn_1M = format_ORC_HF(orc_noroughdyn_1M_file, 'green', 'noirr_noroughdyn')

orc_noroughdyn_1M

In [ ]:
#select sites and golden days for nouroughdyn sim
cen_lmdz_noroughdyn = sel_closest(lmdz_noroughdyn, lon=Cendrosa_lon,lat=Cendrosa_lat)
cen_orc_noroughdyn = sel_closest(orc_noroughdyn, lon=Cendrosa_lon,lat=Cendrosa_lat)
els_lmdz_noroughdyn = sel_closest(lmdz_noroughdyn, lon=ElsPlans_lon,lat=ElsPlans_lat)
els_orc_noroughdyn = sel_closest(orc_noroughdyn, lon=ElsPlans_lon,lat=ElsPlans_lat)
cen_lmdz_noroughdyn.attrs['name']='La Cendrosa no_irr_noroughdyn'
cen_orc_noroughdyn.attrs['name']='La Cendrosa no_irr_noroughdyn'
els_lmdz_noroughdyn.attrs['name']='Els Plans no_irr_noroughdyn'
els_orc_noroughdyn.attrs['name']='Els Plans no_irr_noroughdyn'
#add attr linestyle - to nouroughdyn datasets
cen_lmdz_noroughdyn.attrs['linestyle']='-'
cen_orc_noroughdyn.attrs['linestyle']='-'
els_lmdz_noroughdyn.attrs['linestyle']='-'
els_orc_noroughdyn.attrs['linestyle']='-'

gold_cen_lmdz_noroughdyn = filter_xarray_by_timestamps(cen_lmdz_noroughdyn, start_time, end_time)
gold_cen_orc_noroughdyn = filter_xarray_by_timestamps(cen_orc_noroughdyn, start_time, end_time)
gold_els_lmdz_noroughdyn = filter_xarray_by_timestamps(els_lmdz_noroughdyn, start_time, end_time)
gold_els_orc_noroughdyn = filter_xarray_by_timestamps(els_orc_noroughdyn, start_time, end_time)

# Load obs files and edit datasets

## RS

In [ ]:
RS_days_list=['1507','1607','1707','2007','2107','2207','2707']

cendrosa_dir='../../../obs/LIAISE/LIAISE_LA-CENDROSA_CNRM_RS_L2'
cen_files_list=[cendrosa_files_1507,cendrosa_files_1607,cendrosa_files_1707,cendrosa_files_2007,cendrosa_files_2107,cendrosa_files_2207,cendrosa_files_2707]
cen_times_list=[cen_1507_times,cen_1607_times,cen_1707_times,cen_2007_times,cen_2107_times,cen_2207_times,cen_2707_times]

els_dir='../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_radiosondes_L1'
els_files_list=[els_files_1507,els_files_1607,els_files_1707,els_files_2007,els_files_2107,els_files_2207,els_files_2707]
els_times_list=[els_1507_times,els_1607_times,els_1707_times,els_2007_times,els_2107_times,els_2207_times,els_2707_times]

In [ ]:
cen_RS={}
els_RS={}
for i in range(len(cen_files_list)):
    # print(RS_days_list[i])
    cen_RS_day=RS_extract_full_day(cen_files_list[i], cen_times_list[i], cendrosa_dir, format_Cendrosa_RS)
    els_RS_day=RS_extract_full_day(els_files_list[i], els_times_list[i], els_dir, format_ElsPlans_RS)
    cen_RS[RS_days_list[i]]=cen_RS_day
    els_RS[RS_days_list[i]]=els_RS_day

## Masts measurements

In [ ]:
def format_Cendrosa_MTO(filename):
    ds = xr.open_mfdataset(filename)
    ds = ds.assign_coords(time_decimal=ds.time.dt.hour + ds.time.dt.minute / 60)
    #add name and plot color
    ds.attrs['name'] = 'Cendrosa_MTO'
    ds.attrs['plot_color'] = 'black'
    ds.attrs['linestyle']='-'

    rename_dict = {
        #lmdz vars
        'ta_2':'t2m',
        'shf_1':'sens',
        'lhf_1':'flat',
        'hur_2':'rh2m',
        'hus_2':'q2m',
        'ws_2':'wind_speed_10m',
        'wd_2':'wind_direction_10m',
        'pa':'psol',
        'swup':'SWupSFC',
        'lwup':'LWupSFC',
        'swd':'SWdnSFC',
        'lwd':'LWdnSFC',
        'rain_cumul':'precip',

        'ta_3':'t10m',
        'ta_4':'t25m',
        'ta_5':'t50m',
        'hur_3':'rh10m',
        'hur_4':'rh25m',
        'hur_5':'rh50m',
        'hus_3':'q10m',
        'hus_4':'q25m',
        'hus_5':'q50m',
        'shf_2':'sens25m',
        'shf_3':'sens50m',
        'lhf_2':'flat50m',
        
        #orc vars
        'soil_temp_1':'soil_temp_06',
        'soil_temp_2':'soil_temp_12',
        'soil_heat_flux':'Qg',

    }
    ds = ds.rename(rename_dict)

    #change temperatures to K
    ds['t2m'] = ds['t2m'] + 273.15
    ds['t2m'].attrs['units'] = 'K'
    ds['t10m'] = ds['t10m'] + 273.15
    ds['t10m'].attrs['units'] = 'K'
    ds['t25m'] = ds['t25m'] + 273.15
    ds['t25m'].attrs['units'] = 'K'
    ds['t50m'] = ds['t50m'] + 273.15
    ds['t50m'].attrs['units'] = 'K'
    #change sign of Qg
    ds['Qg'] = -ds['Qg']
    #make netrad fluxes
    ds['lwnet'] = ds['LWdnSFC'] - ds['LWupSFC']
    ds['lwnet'].attrs['units'] = 'W/m2'
    ds['swnet'] = ds['SWdnSFC'] - ds['SWupSFC']
    ds['swnet'].attrs['units'] = 'W/m2'
    #humidity in top10cm
    alpha_SM=0.5
    ds['mrsos']=(alpha_SM * ds['soil_moisture_1'] + (1-alpha_SM) * ds['soil_moisture_2']) * 100
    ds['mrsos'].attrs['units'] = 'mm'

    ds['soil_moisture_1'] = ds['soil_moisture_1'] * 100
    ds['soil_moisture_2'] = ds['soil_moisture_2'] * 100
    ds['soil_moisture_3'] = ds['soil_moisture_3'] * 100

    # #precip in mm/d (initialy in mm/30mn)
    # ds['precip'] = ds['precip'] * 2 * 24
    # ds['precip'].attrs['units'] = 'mm/d'

    return(ds)

In [ ]:
filename = '../../../obs/LIAISE/LIAISE_LA-CENDROSA_CNRM_MTO-FLUX-30MIN_L2/LIAISE_LA-CENDROSA_CNRM_MTO-FLUX-30MIN_L2_2021-07-21_V3.nc'
day1_cen_mto = format_Cendrosa_MTO(filename)

filename = '../../../obs/LIAISE/LIAISE_LA-CENDROSA_CNRM_MTO-FLUX-30MIN_L2/LIAISE_LA-CENDROSA_CNRM_MTO-FLUX-30MIN_L2_2021-07-22_V3.nc'
day2_cen_mto = format_Cendrosa_MTO(filename)

#concatenate both ds in gold_cen_mto
gold_cen_mto = xr.concat([day1_cen_mto,day2_cen_mto],dim='time')
gold_cen_mto

In [ ]:
#load all July obs in one ds
cen_mto_filename='../../../obs/LIAISE/LIAISE_LA-CENDROSA_CNRM_MTO-FLUX-30MIN_L2/LIAISE_LA-CENDROSA_CNRM_MTO-FLUX-30MIN_L2_2021-07*_V3.nc'
cen_mto = format_Cendrosa_MTO(cen_mto_filename)
cen_mto

In [ ]:
def format_ElsPlans_MTO(ds, start_day):
    #add name and plot color
    ds.attrs['name'] = 'ElsPlans_MTO'
    ds.attrs['plot_color'] = 'black'
    ds.attrs['linestyle']='--'

    rename_dict = {
        'HOUR_time':'time',
        #lmdz vars
        'PRES_subsoil':'psol',
        'TEMP_2m':'t2m',
        'RHUM_2mB':'rh2m',
        'UTOT_10m':'wind_speed_10m',
        'DIR_10m':'wind_direction_10m',
        'SWUP_rad':'SWupSFC',
        'LWUP_rad':'LWupSFC',
        'SWDN_rad':'SWdnSFC',
        'LWDN_rad':'LWdnSFC',
        'RAIN_subsoil' : 'precip',
        'TEMP_50m': 't50m',
        'RHUM_50m': 'rh50m',
        'TEMP_25m': 't25m',
        'RHUM_25m': 'rh25m',
        'TEMP_10m': 't10m',

        #orc vars
        'SFLXA_subsoil':'Qg',
        'ST01_subsoil':'soil_temp_01',
        'ST04_subsoil':'soil_temp_03',
        'ST10_subsoil':'soil_temp_12',
    }
    ds = ds.rename(rename_dict)
    # ds = ds.assign_coords(time_decimal=ds.time.dt.hour + ds.time.dt.minute / 60)

    #convert time to hour:mn:s
    ds['time'] = pd.to_datetime(start_day) + pd.to_timedelta(ds['time'], unit='h')

    #add coord time_decimal for diurnal cycle
    ds = ds.assign_coords(time_decimal=ds['time'].dt.hour + ds['time'].dt.minute / 60)

    #remove all values of 1e11
    ds = ds.where(ds != 1e11)
    # change temperatures to K
    ds['t2m'] = ds['t2m'] + 273.15
    ds['t2m'].attrs['units'] = 'K'
    ds['t10m'] = ds['t10m'] + 273.15
    ds['t10m'].attrs['units'] = 'K'
    ds['t25m'] = ds['t25m'] + 273.15
    ds['t25m'].attrs['units'] = 'K'
    ds['t50m'] = ds['t50m'] + 273.15
    ds['t50m'].attrs['units'] = 'K'
    #get turbulent fluxes
    # Latent heat of vaporization of water in J/kg
    lambda_v = 2.5e6
    ds['flat'] = ds['WQ_2m'] * lambda_v
    ds['flat'].attrs['units'] = 'W/m2'
    # air density in kg/m3 and specific heat capacity of air in J/kg/K
    rho=1.225
    cp=1004.67
    ds['sens'] = ds['WT_2m'] * rho * cp
    ds['sens'].attrs['units'] = 'W/m2'
    # make net rad fluxes
    ds['lwnet'] = ds['LWdnSFC'] - ds['LWupSFC']
    ds['lwnet'].attrs['units'] = 'W/m2'
    ds['swnet'] = ds['SWdnSFC'] - ds['SWupSFC']
    ds['swnet'].attrs['units'] = 'W/m2'
    #change sign of Qg
    ds['Qg'] = -ds['Qg']
    #humidity in top10cm
    # alpha_SM=0.5
    # ds['mrsos']=(alpha_SM * ds['SM02_subsoil'] + (1-alpha_SM) * ds['SM10_subsoil']) * 10
    ds['mrsos'] = ds['SM10_subsoil']
    ds['mrsos'].attrs['units'] = 'mm'

    return(ds)

In [ ]:
filename =  '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2_20210721_V4.0.dat'
ds = dat_to_xarray(filename, 152, 153)
day1_els_mto = format_ElsPlans_MTO(ds, start_day='2021-07-21')
# day1_els_mto

filename =  '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2_20210722_V4.0.dat'
ds = dat_to_xarray(filename, 152, 153)
day2_els_mto = format_ElsPlans_MTO(ds, start_day='2021-07-22')
# day2_els_mto

#concatenate
gold_els_mto = xr.concat([day1_els_mto,day2_els_mto],dim='time')
gold_els_mto

In [ ]:
july_days=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
for day in july_days:
    filename =  '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2_202107{}_V4.0.dat'.format(day)
    ds = dat_to_xarray(filename, 152, 153)
    day_els_mto = format_ElsPlans_MTO(ds, start_day='2021-07-{}'.format(day))
    if day=='01':
        els_mto=day_els_mto
    else:
        els_mto=xr.concat([els_mto,day_els_mto],dim='time')
els_mto

In [ ]:
# filename =  '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2_20210722_V4.0.dat'
# ds = dat_to_xarray(filename, 152, 153)
# els_mto = format_ElsPlans_MTO(ds, start_day='2021-07-22')

# Maps

In [ ]:
file=orc_irr_1M_file
var='evap'

lat_min=41.35
lat_max=41.82
lon_min=0.45
lon_max=1.35

vmin=00.0
vmax=7000
cmap=reds
# (file, var, vmin, vmax, cmap)=(orc_irr_1M_file, 'irrigation', 0, 0.2, wet)

plot_ICO_from_netcdf(file, var, 
                     cmap=cmap,
                    #  lon_min=lon_min, lon_max=lon_max, lat_min=lat_min, lat_max=lat_max,
                    #  vmin=vmin, vmax=vmax, 
                    #  title='Irrigation (July 2021, mm/d)'
    )

ax = plt.gca()
ax.plot(Cendrosa_lon, Cendrosa_lat, 'go', markersize=10, transform=ccrs.Geodetic())
ax.plot(ElsPlans_lon, ElsPlans_lat, 'go', markersize=10, transform=ccrs.Geodetic())
ax.plot(Cendrosa_Longitude, Cendrosa_Latitude, 'ro', markersize=10, transform=ccrs.Geodetic())
ax.plot(ElsPlans_Longitude, ElsPlans_Latitude, 'ro', markersize=10, transform=ccrs.Geodetic())

# Time series

In [ ]:
var='t2m'
modorc=False
timeslot='gold' 
# timeslot='gold'
if timeslot=='gold':
    if modorc:
        ds1=gold_cen_orc_irr
        ds2=gold_cen_orc_noirr
        ds3=gold_els_orc_irr
        ds4=gold_els_orc_noirr
        ds7=gold_cen_orc_noroughdyn
        ds8=gold_els_orc_noroughdyn
    else:
        ds2=gold_cen_lmdz_noirr
        ds1=gold_cen_lmdz_irr
        ds3=gold_els_lmdz_irr
        ds4=gold_els_lmdz_noirr
        ds7=gold_cen_lmdz_noroughdyn
        ds8=gold_els_lmdz_noroughdyn
    ds5=gold_cen_mto
    ds6=gold_els_mto
elif timeslot=='july':
    if modorc:
        ds1=cen_orc_irr
        ds2=cen_orc_noirr
        ds3=els_orc_irr
        ds4=els_orc_noirr
        ds7=cen_orc_noroughdyn
        ds8=els_orc_noroughdyn
    else:
        ds2=cen_lmdz_noirr
        ds1=cen_lmdz_irr
        ds3=els_lmdz_irr
        ds4=els_lmdz_noirr
        ds7=cen_lmdz_noroughdyn
        ds8=els_lmdz_noroughdyn
    ds5=cen_mto
    ds6=els_mto


ds_list=[ds1, ds2, ds5]#, ds7]
# ds_list=[ds1, ds2, ds3, ds4, ds5, ds6]

vmin=-0
vmax=40
vmin, vmax=None, None
# title='{} ({})'.format(var, ds1[var].attrs['units'])
title=None

time_series_ave(ds_list, var, ds_colors=True, 
                # ds_linestyle=True, 
                title=title, vmin=vmin, vmax=vmax)

vmin, vmax=None, None
ds_list=[ds3, ds4, ds6]#, ds8]
# ds_list=[ds4, ds8]
time_series_ave(ds_list, var, ds_colors=True, ds_linestyle=True, title=title, vmin=vmin, vmax=vmax)

In [ ]:
#diurnal cycles
var='q2m'
modorc=False
if modorc:
    ds1=cen_orc_irr
    ds2=cen_orc_noirr
    ds3=els_orc_irr
    ds4=els_orc_noirr
    ds7=cen_orc_noroughdyn
    ds8=els_orc_noroughdyn
else:
    ds2=cen_lmdz_noirr
    ds1=cen_lmdz_irr
    ds3=els_lmdz_irr
    ds4=els_lmdz_noirr
    ds7=cen_lmdz_noroughdyn
    ds8=els_lmdz_noroughdyn
ds5=cen_mto
ds6=els_mto


ds_list=[ds1, ds2, ds5]#, ds7]
# ds_list=[ds1, ds2, ds3, ds4, ds5, ds6]
# ds_list=[ ds2, ds7]

vmin=6.5
vmax=12
# vmin, vmax=None, None
title='{} ({})'.format(var, ds1[var].attrs['units'])
# title='Soil temperature 6cm (°C)'

diurnal_cycle_ave(ds_list, var, ds_colors=True, 
                # ds_linestyle=True, 
                title=title, 
                vmin=vmin, vmax=vmax
                )

# vmin, vmax=None, None
# ds_list=[ds3, ds4, ds6]#, ds8]
# # ds_list=[ds4, ds8]
# diurnal_cycle_ave(ds_list, var, ds_colors=True, title=title, 
#                 #   ds_linestyle=True,
#                   vmin=vmin, vmax=vmax
#                   )   

In [ ]:
#plot similar vars in obs to see how they compare
ds=cen_mto
vars=['soil_moisture_1','soil_moisture_2','soil_moisture_3', 'mrsos']
# vars=['mrsos','precip']
plotvars=[ds[var] for var in vars]+[cen_orc_irr['mrsos']]
labels=vars + ['mrsos_ORC_irr']
time_series(plotvars, labels, title='Cendrosa MTO')

ds=els_mto
vars=['SM02_subsoil','SM10_subsoil','PR10_subsoil','PR20_subsoil']
plotvars=[ds[var] for var in vars]+[els_orc_irr['mrsos']]
labels=vars+ ['mrsos_ORC_irr']
time_series(plotvars, labels, title='Els Plans MTO')

# Vertical profiles

In [ ]:
var='tke'
ds1=day1_els_lmdz_irr
ds2=day1_els_lmdz_noirr
ds3=day1_cen_lmdz_irr
ds4=day1_cen_lmdz_noirr

ds_list=[ds1, ds2, ds3, ds4]
# ds_list=[ds3, ds4]
ds_list=[ds1]
# ds_list=[cen_lmdz_irr]

# obs_ds_list=[Cendrosa_RS]
obs_ds_list=None

altmax=200
title=None

# profile_preslevs_local(ds_list, var, title=title, preslevelmax=preslevelmax)
profile_altitude_local_mean(ds_list, var, title=title, altmax=altmax,
                            obs_ds_list=obs_ds_list,
                            nbins=None,
                            substract_gl=True,
                            )

In [ ]:
ds1=day1_els_lmdz_irr
ds2=day1_els_lmdz_noirr
ds3=cen_lmdz_irr
ds4=cen_lmdz_noirr

ds_list, obs_dict=[day1_els_lmdz_irr, day1_els_lmdz_noirr], els_RS_2107
ds_list, obs_dict=[day1_cen_lmdz_irr, day1_cen_lmdz_noirr], cen_RS_2107

# ds_list, obs_dict=[day2_els_lmdz_irr, day2_els_lmdz_noirr], els_RS_2207
ds_list, obs_dict=[day2_cen_lmdz_irr, day2_cen_lmdz_noirr], cen_RS_2207

# ds_list, obs_dict =[day1_cen_lmdz_irr], None
# obs_dict=None

altmax=2500
title=None

# times=els_2107_times+0.5
# times=[x + 0.5 for x in cen_2107_times]
times=[12.25]

var='theta'
var, xmin, xmax = 'theta', 298, 314
# var, xmin, xmax = 'ovap', 2, 14
# var, xmin, xmax = 'wind_speed', 0, 9
# var, xmin, xmax = 'wind_direction', 0, 360
# xmin, xmax = None, None

# profile_preslevs_local(ds_list, var, title=title, preslevelmax=preslevelmax)
profile_altitude_multipletimes_obs(ds_list, obs_dict, var, times, 
                                 # title=title, 
                                 altmax=altmax, xmin=xmin, xmax=xmax, 
                                 # nbins=5,
                                 substract_gl=True,
                                 simfreq='30mn'
                                )

# Site caracterization

In [ ]:
cen_orc_irr_1M=sel_closest(orc_irr_1M, Cendrosa_lon, Cendrosa_lat)
cen_orc_noirr_1M=sel_closest(orc_noirr_1M, Cendrosa_lon, Cendrosa_lat)
els_orc_irr_1M=sel_closest(orc_irr_1M, ElsPlans_lon, ElsPlans_lat)
els_orc_noirr_1M=sel_closest(orc_noirr_1M, ElsPlans_lon, ElsPlans_lat)

#rename ds according to site and irr
cen_orc_irr_1M.attrs['name']='La Cendrosa irr'
cen_orc_noirr_1M.attrs['name']='La Cendrosa no_irr'
els_orc_irr_1M.attrs['name']='Els Plans irr'
els_orc_noirr_1M.attrs['name']='Els Plans no_irr'

cen_orc_irr_1M

In [ ]:
cen_lmdz_irr_1M=sel_closest(lmdz_irr_1M, Cendrosa_lon, Cendrosa_lat)
cen_lmdz_noirr_1M=sel_closest(lmdz_noirr_1M, Cendrosa_lon, Cendrosa_lat)
els_lmdz_irr_1M=sel_closest(lmdz_irr_1M, ElsPlans_lon, ElsPlans_lat)
els_lmdz_noirr_1M=sel_closest(lmdz_noirr_1M, ElsPlans_lon, ElsPlans_lat)

#rename ds according to site and irr
cen_lmdz_irr_1M.attrs['name']='La Cendrosa irr'
cen_lmdz_noirr_1M.attrs['name']='La Cendrosa no_irr'
els_lmdz_irr_1M.attrs['name']='Els Plans irr'
els_lmdz_noirr_1M.attrs['name']='Els Plans no_irr'

cen_lmdz_irr_1M

In [ ]:
def process_list(input_list):
    print( [round(x * 100, 1) for x in input_list])

In [ ]:
process_list(cen_orc_irr_1M['soiltile'].values)
process_list(cen_orc_irr_1M['maxvegetfrac'].values[0])
process_list(cen_orc_irr_1M['vegetfrac'].values[0])

In [ ]:
process_list(els_orc_irr_1M['soiltile'].values)
process_list(els_orc_irr_1M['maxvegetfrac'].values[0])
process_list(els_orc_irr_1M['vegetfrac'].values[0])

In [ ]:
def round_value(val):
    if isinstance(val, xr.DataArray):
        return round(float(val.values), 1)
    elif isinstance(val, (int, float)):
        return round(val, 2)
    return val

In [ ]:
orc_varlist=['tot_bare_soil', 'LAImean', 'albedo_glob',
             'irrig_frac', 'aei_sw', 'root_deficit', 'netirrig','irrigation',
             'mrsos', 'temp_sol', 'rain']
orc_ds_list=[cen_orc_irr_1M, els_orc_irr_1M]

rows = []
for ds in orc_ds_list:
    row_data = {}
    for var in orc_varlist:
        row_data[var] = ds[var].values[0]
    rows.append(row_data)

df = pd.DataFrame(rows)
df=df.round(2)
df.index = [ds.attrs['name'] for ds in orc_ds_list]

df

In [ ]:
lmdz_varlist=['ground_level','psol','t2m','flat', 'sens',
               'SWdnSFC', 'LWdnSFC', 'SWupSFC', 'LWupSFC']
lmdz_ds_list=[cen_lmdz_irr_1M, els_lmdz_irr_1M]

rows = []
for ds in lmdz_ds_list:
    row_data = {}
    for var in lmdz_varlist:
        row_data[var] = round(ds[var].values[0],1)
        # print (var, row_data[var])
    rows.append(row_data)

df = pd.DataFrame(rows)
df = df.map(round_value)
df.index = [ds.attrs['name'] for ds in lmdz_ds_list]

df

In [ ]:
ds=els_lmdz_noirr_1M
# ds=day2_els_lmdz_irr
(ds['geoph']- ds['ground_level']).values